In [1]:
from ipynb.fs.full.sim import Simulator
from ipynb.fs.full.traffic import Frame
from ipynb.fs.full.logger import Logger
from ipynb.fs.full.topology import BasicSwitch, IngressPort, EgressPort

In [2]:
class FRERSplit:
    def __init__(self, split_config: dict):
        """
        split_config:
            stream_id -> list of egress_port_ids
        """
        self.split_config = split_config

    def apply(self, frame: Frame):
        ports = self.split_config.get(frame.stream_id)
        if not ports:
            return [(frame, None)]

        replicas = []
        for i, port_id in enumerate(ports):
            replica = Frame(
                frame_id=frame.frame_id,
                stream_id=frame.stream_id,
                size=frame.size,
                creation_time=frame.creation_time,
                metadata={**frame.metadata, "frer_replica": i}
            )
            replicas.append((replica, port_id))

        return replicas

In [3]:
class FRERMerge:
    def __init__(self, expire_time: float):
        self.expire_time = expire_time
        self.seen = {}  # stream_id -> {frame_id: first_seen_time}

    def apply(self, frame: Frame, now: float) -> bool:
        stream_seen = self.seen.setdefault(frame.stream_id, {})

        # Cleanup expired entries
        for fid, ts in list(stream_seen.items()):
            if now - ts > self.expire_time:
                del stream_seen[fid]

        if frame.frame_id in stream_seen:
            return False  # duplicate

        stream_seen[frame.frame_id] = now
        return True

In [4]:
class FRERSwitch(BasicSwitch):
    def __init__(self, name, simulator, logger, routing_table, frer_split=None, frer_merge=None):
        super().__init__(name, simulator, logger, routing_table)
        self.frer_split = frer_split
        self.frer_merge = frer_merge

    def process(self, frame: Frame, ingress_port: IngressPort):
        # Merge
        if self.frer_merge:
            if not self.frer_merge.apply(frame, self.sim.time):
                self.logger.log(self.sim.time, self.name, "FRER_DROP", frame, ingress_port.port_id)
                return

        # Split
        if self.frer_split:
            replicas = self.frer_split.apply(frame)   # → list[(Frame, port_id)]
        else:
            replicas = [(frame, None)]

        for replica, explicit_port in replicas:
            if explicit_port is not None:
                out_ports = [explicit_port]
            else:
                out_ports = self.routing_table.get(replica.stream_id, [])

            for port_id in out_ports:
                if port_id not in self.egress_ports:
                    self.logger.log(self.sim.time, self.name, "DROP", replica, port_id)
                    continue

                self.queues[port_id].append(replica)

                if not self.sending[port_id]:
                    self._send_next(port_id)